In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
#import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
orig_data = pd.read_csv("data/test_2012.csv")
data = orig_data.copy()

In [3]:
data_orig_cols = list(orig_data.columns) # original columns, with target PES1 on the end

# Feature Engineering

In [4]:
### Goal: segment intuitively important features; standardize yes/no questions to binary

# Segment HUFAMINC (family income)
data['low_fam_inc'] = data.HUFAMINC.apply(lambda x: x >= 1 and x <= 11).astype(int)
data['med_fam_inc'] = data.HUFAMINC.apply(lambda x: x >= 12 and x <= 14).astype(int)
data['high_fam_inc'] = data.HUFAMINC.apply(lambda x: x >= 15).astype(int)

# Segment HRNUMHOU (number of household members)
data['one_person_hh'] = (data.HRNUMHOU == 1).astype(int)
data['two_person_hh'] = (data.HRNUMHOU == 2).astype(int)
data['multi_person_hh'] = (data.HRNUMHOU > 2).astype(int)

# Segment HUBUS (business/farm owner?)
data['farm_bus_hh'] = (data.HUBUS == 1).astype(int)
data['non_farm_bus_hh'] = (data.HUBUS == 2).astype(int)

# Segment GEREG (region)
data['northeast'] = (data.GEREG == 1).astype(int)
data['midwest'] = (data.GEREG == 2).astype(int)
data['south'] = (data.GEREG == 3).astype(int)
data['west'] = (data.GEREG == 4).astype(int)

# Segment PEAGE (age)
data['too_young'] = data.PEAGE.apply(lambda x: x >= 0 and x < 18).astype(int)
data['youth'] = data.PEAGE.apply(lambda x: x >= 18 and x < 40).astype(int)
data['middle_age'] = data.PEAGE.apply(lambda x: x >= 40 and x < 65).astype(int)
data['elderly'] = data.PEAGE.apply(lambda x: x >= 65).astype(int)

# Segment PEEDUCA (highest level of school completed or degree received)
data['hs_dropout'] = data.PEEDUCA.apply(lambda x: x < 39).astype(int)
data['hs_grad'] = data.PEEDUCA.apply(lambda x: x >= 39).astype(int)
data['bachelor_deg'] = data.PEEDUCA.apply(lambda x: x >= 43).astype(int)
data['adv_deg'] = data.PEEDUCA.apply(lambda x: x >= 44).astype(int)

# Segment PTDTRACE (race)
data['white_race'] = data.PTDTRACE.apply(lambda x: x == 1).astype(int)
data['black_race'] = data.PTDTRACE.apply(lambda x: x == 2).astype(int)
data['asian_race'] = data.PTDTRACE.apply(lambda x: x == 4).astype(int)
data['mixed_race'] = data.PTDTRACE.apply(lambda x: x >= 6).astype(int)

# Segment PENATVTY (country of birth)
data['us_born'] = data.PENATVTY.apply(lambda x: x == 57).astype(int)
data['us_territory_born'] = data.PENATVTY.apply(lambda x: x in [66, 73, 78, 96]).astype(int)
data['born_outside_us'] = data.PENATVTY.apply(lambda x: x == 555).astype(int)

# Segment PRINUSYR (immigrant entry year)
data['imm_last_10'] = data.PRINUSYR.apply(lambda x: x >= 16).astype(int)
data['imm_last_20'] = data.PRINUSYR.apply(lambda x: x >= 11 and x < 16).astype(int)
data['imm_before_1988'] = data.PRINUSYR.apply(lambda x: x >= 1 and x < 11).astype(int)

# Segment PUABSOT (did you have a job last week?)
data['employed'] = data.PUABSOT.apply(lambda x: x == 1).astype(int)
data['unemployed'] = data.PUABSOT.apply(lambda x: x == 2).astype(int) # This does not include retired; only unemployed

# Segment PEIO1COW (employer type)
data['fed_employed'] = data.PEIO1COW.apply(lambda x: x in [1, 2, 3]).astype(int)
data['priv_employed'] = data.PEIO1COW.apply(lambda x: x in [4, 5]).astype(int)
data['self_employed'] = data.PEIO1COW.apply(lambda x: x in [6, 7]).astype(int)

# Segment PRCHLD (has own children)
data['parent'] = data.PRCHLD.apply(lambda x: x >= 1).astype(int)

# Segment PEAFWHN1 (military service period)
data['ww2_vet'] = data.PEAFWHN1.apply(lambda x: x == 8).astype(int)
data['vietnam_vet'] = data.PEAFWHN1.apply(lambda x: x == 4).astype(int)
data['korea_vet'] = data.PEAFWHN1.apply(lambda x: x == 6).astype(int)

# 0/1 PESEX (sex)
data['male'] = data.PESEX.apply(lambda x: x == 1).astype(int)
data['female'] = data.PESEX.apply(lambda x: x == 2).astype(int)

# 0/1 PEHSPNON (hispanic?)
data['hispanic'] = data.PEHSPNON.apply(lambda x: x == 1).astype(int)
data['nonhispanic'] = data.PEHSPNON.apply(lambda x: x == 2).astype(int)

# 0/1 PURETOT (retired?)
data['retired'] = data.PURETOT.apply(lambda x: x == 1).astype(int)
data['not_retired'] = data.PURETOT.apply(lambda x: x >= 2).astype(int) # 3 indicates they just retired; treat as not retired

# 0/1 PEGRPROF (any grad/prof school courses for credit since bachelor's?)
data['grad_courses_since_bach'] = data.PEGRPROF.apply(lambda x: x == 1).astype(int)
data['no_grad_courses_since_bach'] = data.PEGRPROF.apply(lambda x: x == 2).astype(int)

# Drop allocation flag features; handle missing values

In [5]:
# Drop allocation flag features
allocation_flag_feats = [col for col in data.columns if col[:2] == 'PX']
data.drop(allocation_flag_feats, axis=1, inplace=True)

# Replace negative values (all forms of N/A) with NaN
for feat in data.columns:
    data[feat] = data[feat].apply(lambda x: np.NaN if x < 0 else x)
    
# Replace NaN with the mean of the column
for feat in data.columns:
    data[feat].fillna(data[feat].mean(), inplace=True)
    
# Check for columns that are all NaN, and delete them
all_nan_cols = []
for feat in data.columns:
    if np.all(np.isnan(data[feat])):
        all_nan_cols.append(feat)
data.drop(all_nan_cols, axis=1, inplace=True)

In [6]:
pes1 = data['PES1']
data.drop('PES1', axis=1, inplace=True)
data['PES1'] = pes1

KeyError: 'PES1'

In [7]:
data.columns

Index(['id', 'HRMONTH', 'HRYEAR4', 'HURESPLI', 'HUFINAL', 'HUSPNISH',
       'HETENURE', 'HEHOUSUT', 'HETELHHD', 'HETELAVL',
       ...
       'vietnam_vet', 'korea_vet', 'male', 'female', 'hispanic', 'nonhispanic',
       'retired', 'not_retired', 'grad_courses_since_bach',
       'no_grad_courses_since_bach'],
      dtype='object', length=329)

# Write out datasets

In [9]:
# Move PES1 (target feature) to the end of the dataframe
pes1 = data['PES1']
data.drop('PES1', axis=1, inplace=True)
data['PES1'] = pes1

# Make a dataframe of just the engineered features
data_eng_feats_only = data[data.columns[-48:]]



In [186]:
non_eng_feats = [col for col in data.columns if col.isupper()]

# Make a dataframe with no engineered features, just dropped allocation flags and NaNs removed
data_orig_clean = data[['id'] + non_eng_feats]

In [170]:
# Check that columns look right before writing out
data.columns

Index(['id', 'HRMONTH', 'HRYEAR4', 'HURESPLI', 'HUFINAL', 'HUSPNISH',
       'HETENURE', 'HEHOUSUT', 'HETELHHD', 'HETELAVL',
       ...
       'korea_vet', 'male', 'female', 'hispanic', 'nonhispanic', 'retired',
       'not_retired', 'grad_courses_since_bach', 'no_grad_courses_since_bach',
       'PES1'],
      dtype='object', length=326)

In [172]:
# Same for the eng-feats-only dataframe
data_eng_feats_only.columns

Index(['low_fam_inc', 'med_fam_inc', 'high_fam_inc', 'one_person_hh',
       'two_person_hh', 'multi_person_hh', 'farm_bus_hh', 'non_farm_bus_hh',
       'northeast', 'midwest', 'south', 'west', 'too_young', 'youth',
       'middle_age', 'elderly', 'hs_dropout', 'hs_grad', 'bachelor_deg',
       'adv_deg', 'white_race', 'black_race', 'asian_race', 'mixed_race',
       'us_born', 'us_territory_born', 'born_outside_us', 'imm_last_10',
       'imm_last_20', 'imm_before_1988', 'employed', 'unemployed',
       'fed_employed', 'priv_employed', 'self_employed', 'parent', 'ww2_vet',
       'vietnam_vet', 'korea_vet', 'male', 'female', 'hispanic', 'nonhispanic',
       'retired', 'not_retired', 'grad_courses_since_bach',
       'no_grad_courses_since_bach', 'PES1'],
      dtype='object')

In [188]:
# Same for the eng-feats-only dataframe
data_orig_clean.columns

Index(['id', 'HRMONTH', 'HRYEAR4', 'HURESPLI', 'HUFINAL', 'HUSPNISH',
       'HETENURE', 'HEHOUSUT', 'HETELHHD', 'HETELAVL',
       ...
       'PEMOMTYP', 'PECOHAB', 'PEDISEAR', 'PEDISEYE', 'PEDISREM', 'PEDISPHY',
       'PEDISDRS', 'PEDISOUT', 'PRDISFLG', 'PES1'],
      dtype='object', length=279)

In [189]:
# Write out new data sets
data_eng_feats_only.to_csv('train_2008_eng_feats_only.csv', index=False)
data.to_csv('train_2008_with_eng_feats_no_allo_flags.csv', index=False)
data_orig_clean.to_csv('train_2008_clean_no_eng_feats.csv', index=False)

# Remove misleading features (mostly codes)

starting with data that has engineered features, negative values set to the mean of the feature, and allocation flags removed.

In [8]:
# first list is from eyeballing; second is list of "weight" features that are just used for statistical tallying
bad_cols = ['QSTNUM', 'GESTFIPS', 'GESTCEN', 'HRMIS', 'GTCBSA'] + [col for col in data.columns if col[-3:] == 'WGT']

# drop 'em
data.drop(bad_cols, axis=1, inplace=True)

In [9]:
### GTCBSASZ indicates size of city of residence - seems useful. Let's dummy it!

# First we need to convert the codes into categorical variables
city_size_map = {
                    0:  'rural',
                    2:  '100_250',
                    3:  '250_500',
                    4:  '500_1000',
                    5:  '1000_2500',
                    6:  '2500_5000',
                    7:  '5000_plus'
                }
data['GTCBSASZ'] = data['GTCBSASZ'].map(city_size_map)

# encode in dummy vars
city_size_dummies = pd.get_dummies(data['GTCBSASZ'], prefix='city_size')

# add these new features to the data
data = pd.concat([data, city_size_dummies], axis=1)

In [10]:
# Drop the original city size feature
data.drop('GTCBSASZ', axis=1, inplace=True)


In [11]:
data.columns

Index(['id', 'HRMONTH', 'HRYEAR4', 'HURESPLI', 'HUFINAL', 'HUSPNISH',
       'HETENURE', 'HEHOUSUT', 'HETELHHD', 'HETELAVL',
       ...
       'not_retired', 'grad_courses_since_bach', 'no_grad_courses_since_bach',
       'city_size_1000_2500', 'city_size_100_250', 'city_size_2500_5000',
       'city_size_250_500', 'city_size_5000_plus', 'city_size_500_1000',
       'city_size_rural'],
      dtype='object', length=323)

# Keep just the selected features

In [12]:
import pickle as pkl

In [13]:
keep_feats = pkl.load(open('keep_feats.pkl', 'rb'))

In [14]:
data = data[keep_feats]

In [16]:
data.to_csv('data/test_cleaned_for_2012_sub.csv', index=False)

In [15]:
len(data.columns)

109

In [17]:
len(orig_data)

82820